# Kolmogorov-Smirnov Goodness-of-fit Test for varying weight mixture models

Margaux Thorez

## 1- Contextualisation

We have studied the theoretical threshold calibration of the Kolmogorov-Smirnov goodness-of-fit test applied to variable weight mixture models. We carried out simulations to assess the relevance of the theoretical thresholds found for Ryzhov's method.

We showed that, for group "l" : 
\begin{align*}
    \mathbb{P}\left(\sqrt{n} \sup\limits_{t\in\mathbb{R}} |\hat{S}_l(t) - S_0(t)| >  s_{\alpha} \right) & \leq \mathbb{P}\left(\sqrt{\frac{n}{2}}\sup\limits_{t\in\mathbb{R}}|(\hat{S}_A(t) - S_A(t))| > \frac{ s_{\alpha}}{2\sqrt{2}|a_l(A)|}\right)\\
    & + \mathbb{P}\left(\sqrt{\frac{n}{2}}\sup\limits_{t\in\mathbb{R}}|(\hat{S}_B(t) - S_B(t))| > \frac{s_{\alpha}}{2\sqrt{2}|a_1(B)|}\right).
\end{align*}

We can now determine the threshold $s_{\alpha}$ that applies in the case of our mixture model. We can choose $s_{\alpha} = \max (s_{\alpha_A}, s_{\alpha_B}) $, with $s_{\alpha_A} = 2\sqrt{2}|a_l(A)|t_{\alpha/2} $ and $s_{\alpha_B} = 2\sqrt{2}|a_l(B)|t_{\alpha/2} $. We use the $t_{\alpha/2}$ threshold from the one-sample Kolmogorov-Smirnov table.


The Kolmogorov-Smirnov tables are available on https://real-statistics.com/statistics-tables/kolmogorov-smirnov-table/

## 2- Libraries import

In [1]:
#Libraries import
#Basic librairies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import *
from scipy import stats
import random
import statistics
import seaborn as sns

#scikit-survival Kaplan-Meier estimator
from sksurv.nonparametric import kaplan_meier_estimator
from datetime import datetime

#Interpolation
from scipy.interpolate import interp1d

#CVXPY for convex optimization problems.
import cvxpy as cp

#Numba import
from numba import njit, vectorize

#Parallelization modules
from sklearn.utils._joblib import Parallel, delayed

#Kolmogorov-Smirnov Test
from scipy.stats import ks_2samp
from scipy.stats import kstest
from scipy.stats import ksone
from scipy.stats import kstwo

#Normality Test
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import norm

import statsmodels.stats.diagnostic

from sklearn.mixture import GaussianMixture

#Notebook
from jyquickhelper import add_notebook_menu
import warnings
warnings.filterwarnings('ignore')

In [2]:
add_notebook_menu()

## 3- Optimized functions

For the law parameter choice, we have to choose the parameter "law" among "exponnential", "pareto", "weibull" and "gamma"

### Survival functions

In [3]:
#Survival functions : returns the survival function of a mixture model for a certain law
def S(x,P1,P2,law,lambd1,lambd2):
    if law == "exponnential":
        S = P1*stats.expon.sf(x, scale=lambd1) + P2*stats.expon.sf(x, scale=lambd2)
    if law == "pareto":
        #Certain parameters of the Pareto law have been fixed in order to obtain a function with characteristics similar to the survival functions found for sick leaves
        S = P1*stats.pareto.sf(x, b=3, loc = -(2/3)*lambd1, scale=(2/3)*lambd1) + P2*stats.pareto.sf(x, b=3, loc=-(2/3)*lambd2, scale=(2/3)*lambd2)
    if law == "weibull":
        #Certain parameters of the Weibull law have been fixed in order to obtain a function with characteristics similar to the survival functions found for sick leaves
        S = P1*stats.weibull_min.sf(x, c=2, scale=lambd1) + P2*stats.weibull_min.sf(x, c=2, scale=lambd2)
    if law == "gamma":
        S = P1*stats.gamma.sf(x, a=lambd1) + P2*stats.gamma.sf(x, a=lambd2)
    return(S)

### Simulation functions

In [4]:
#Function that creates the base sample in a mixture according to the chosen law
def InitializeMatriceX(law, lambd1, lambd2, P_1, size):
    #Matrices X_A1 and X_A2
    if law == "exponnential":
        X_1 = stats.expon.rvs(scale=lambd1, size=size)
        X_2 = stats.expon.rvs(scale=lambd2, size=size)
    if law == "pareto":
        X_1_v1 = stats.pareto.rvs(b=3, size=size)
        X_1 = (2/3)*lambd1*(X_1_v1-1)
        X_2_v1 = stats.pareto.rvs(b=3, size=size)
        X_2 = (2/3)*lambd2*(X_2_v1-1)
    if law == "weibull":
        X_1 = stats.weibull_min.rvs(c=2, scale=lambd1, size=size)
        X_2 = stats.weibull_min.rvs(c=2, scale=lambd2, size=size)    
    if law == "gamma" :
        X_1 = stats.gamma.rvs(a=lambd1, size=size)
        X_2 = stats.gamma.rvs(a=lambd2, size=size)
    #Matrice RA
    R = stats.bernoulli.rvs(P_1, size=size)
    X = R*X_1 + (1-R)*X_2
    return X

In [5]:
#Function to include censorship
@njit #(parallel=True)
def Y_ind_Allocation(Y_ind, Y, C, X):
    for i in range(len(X)):
        if Y[i]<C[i]: 
            Y_ind[i]=True
    return Y_ind

In [6]:
#Function to interpolate survival functions
def S_est_Actualisation(S_est, time, time1, t):
    S_est_res = np.concatenate(([1], S_est,[0]))
    f = interp1d(np.concatenate(([0], time,[t])), S_est_res)
    S_est_res = f(time1)
    return S_est_res

### Kaplan-Meier function of Maiboroda

In [7]:
#Function to count S_X and N_X variables for Maiboroda's estimator
@njit #(parallel=True)
def S_x_N_x_Allocation(Y,Y_ind,X_X,time):
    S_X = np.zeros_like(time)
    N_X = np.zeros_like(time)
    for k, t_k in enumerate(time):
        for i, Y_i in enumerate(Y):
            if Y_i >= t_k:
                S_X[k] += X_X[i]
            if (Y_i <= t_k) and  (Y_ind[i]==True) : 
                N_X[k] += X_X[i]
    return S_X, N_X

In [8]:
#Function to determine Maiboroda's estimator having S_X and N_X
@njit #(parallel=True)
def S_1_mx_Allocation(Y, S_X, N_X, time):
    # Initialisation
    S_1_mx = np.ones_like(time)
    for k, t_k in enumerate(time):
        for i in range(1,k):
            if (Y[i] <= t_k) and (S_X[i] != 0) :
                S_1_mx[k] = S_1_mx[k] * (1 - (N_X[i] - N_X[i-1])/S_X[i])
    return S_1_mx

In [9]:
#Function to smooth a survival function
@njit #(parallel=True)
# direction = 0 or 1
def S_1_est_mx_Threshold(S_1_est_mx, direction):
    S1_res = np.zeros_like(S_1_est_mx)
    #Maximum at 1 and minimum at 0 for the survival function
    for i, item in enumerate(S_1_est_mx):
        if item > 1 :
            S1_res[i] = 1
        elif item < 0 :
            S1_res[i] = 0
        else :
            S1_res[i] = item
    #If direction = 0, we give to S[t] the value of S[t+1] if S[t]<S[t+1]
    if direction == 0 : 
        for i in range(len(S_1_est_mx)-1):
            k = i+1
            while k<len(S_1_est_mx):
                if S1_res[i]<S1_res[k]:
                    S1_res[i] = S1_res[k]
                k+=1
    #If direction = 1, we give to S[t+1] the value of S[t] if S[t]<S[t+1]
    elif direction == 1 :
        for i in range(len(S_1_est_mx)-1):
            k = i+1
            while k < len(S_1_est_mx):
                if S1_res[i]<S1_res[k]:
                    S1_res[k] = S1_res[i]
                k+=1
    return S1_res

In [10]:
#Maiboroda's Kaplan-Meier estimator
@njit #(parallel=True)
def kaplan_meier_mx (Y,Y_ind,X_X,time, direction) :
    #Construction of N_X and S_X
    S_X, N_X = S_x_N_x_Allocation(Y=Y,Y_ind=Y_ind,X_X=X_X,time=time)
    #Initialization of S_1_est_mx
    S_1_est_mx = S_1_mx_Allocation(Y=Y, S_X=S_X, N_X=N_X, time=time)
    return S_1_est_mx 

In [11]:
#Maiboroda's Kaplan-Meier estimator smoothed
@njit #(parallel=True)
def kaplan_meier_mx_smoothed (Y,Y_ind,X_X,time, direction) :
    #Construction of N_X and S_X
    S_X, N_X = S_x_N_x_Allocation(Y=Y,Y_ind=Y_ind,X_X=X_X,time=time)
    #Initialization of S_1_est_mx
    S_1_est_mx = S_1_mx_Allocation(Y=Y, S_X=S_X, N_X=N_X, time=time)
    #S_1_est_mx thresholding
    S_1_est_mx_res = S_1_est_mx_Threshold(S_1_est_mx, direction)
    #Return S_1_est_mx_res if we want the survival function to be linearized, S_1_est_mx otherwise
    return S_1_est_mx_res 

## 4- Threshold determination of the Kolmogorov-Smirnov test for varying weight mixture models

In [12]:
def RunSimulation(law, nA, nB, lambd1, lambd2, P_A1, P_A2, P_B1, P_B2, M_inv, direction):
    #The model argument can be either "Ryzhov" or "Maiboroda"
    P = np.array([[P_A1, P_A2], [P_B1, P_B2]])
    #Samples
    X_A = InitializeMatriceX(law=law, lambd1=lambd1, lambd2=lambd2, P_1=P_A1, size=nA)
    X_B = InitializeMatriceX(law=law, lambd1=lambd1, lambd2=lambd2, P_1=P_B1, size=nB)
    #Kaplan-Meier estimators
    Y_A = X_A 
    Y_B = X_B
    Y_A_ind = np.full(shape=Y_A.shape, fill_value=True)
    Y_B_ind = np.full(shape=Y_B.shape, fill_value=True)
    time_A, S_A_est = kaplan_meier_estimator(Y_A_ind, Y_A)
    time_B, S_B_est = kaplan_meier_estimator(Y_B_ind, Y_B)
    #Initialization of time1, t and M_inv
    time1 = np.concatenate(([0],time_A, time_B))
    time1.sort()
    t=max(time_A[-1],time_B[-1])
    #Interpolation of S_A_est and S_B_est
    S_A_est = S_est_Actualisation(S_est=S_A_est, time=time_A, time1=time1, t=t)
    S_B_est = S_est_Actualisation(S_est=S_B_est, time=time_B, time1=time1, t=t)
    #Determination of S_1_est and S_2_est (following Ryzhov's method)
    S_1_est = M_inv[0][0]*S_A_est + M_inv[0][1]*S_B_est
    S_2_est = M_inv[1][0]*S_A_est + M_inv[1][1]*S_B_est
    #True survival functions
    S_A = S(time1,P_A1,P_A2,law,lambd1,lambd2)
    S_B = S(time1,P_B1,P_B2,law,lambd1,lambd2)    
    #sup computation 
    S_0 = S(time1,1,0,law,lambd1,lambd2)
    
    #Calcul des sup
    KS1 = np.linalg.norm(S_1_est - S_0 , np.inf)
    KSA  = np.linalg.norm(S_A_est - S_A , np.inf)
    KSB  = np.linalg.norm(S_B_est - S_B , np.inf)
    return (KS1, KSA, KSB)

In [13]:
def RunParallelSimulation(law, nA, nB, lambd1, lambd2, P_A1, P_A2, P_B1, P_B2, M_inv, direction, K, NJobs=-1, Verbose=0, PreDispatch='2*n_jobs'):
    
    #Initialization of parallelization parameters
    ParallelSetUp = Parallel(n_jobs=NJobs, verbose=Verbose, pre_dispatch=PreDispatch, prefer="threads")
    
    #RunSimulation
    ComputedBlocks = ParallelSetUp([delayed(RunSimulation)(law=law,nA=nA, nB=nB, lambd1=lambd1, lambd2=lambd2, P_A1=P_A1, P_A2=P_A2, P_B1=P_B1, P_B2=P_B2, M_inv=M_inv, direction = direction) for k in range(K)])

    #Concatenation into array
    List_KS1 = np.array([KS1 for KS1, _ , _   in ComputedBlocks], dtype=np.dtype(object)) # peut etre que dtype array est approprié
    List_KSA = np.array([KSA for _, KSA , _  in ComputedBlocks], dtype=np.dtype(object)) # peut etre que dtype array est approprié
    List_KSB = np.array([KSB for _, _ , KSB  in ComputedBlocks], dtype=np.dtype(object)) # peut etre que dtype array est approprié
  
    #return ComputedBlocks
    return List_KS1, List_KSA , List_KSB 

### Simulation parameters

In [14]:
#Simulations
#Parameters choice (choose among "exponnential", "pareto", "weibull" and "gamma")
law = "exponnential" 

#Sample sizes (we have chosen nA = nB but it is possible to change these parameters)
n = 10000
nA = round(n/2)
nB = n - nA

#Mixture parameters
lambd1 = 50
lambd2 = 50

#Weight selection (note that we must have P_A1 + P_A2 = 1 and P_B1 + P_B2 = 1)
P_A1 = 0.7
P_A2 = 0.3
P_B1 = 0.1
P_B2 = 0.9

#Threshold direction
direction = 1

In [15]:
#Ryzhov :
M = np.array([[P_A1, P_A2],[P_B1, P_B2]])
M_inv = np.linalg.inv(M)
print(M_inv)

[[ 1.5        -0.5       ]
 [-0.16666667  1.16666667]]


In [16]:
RunSimulation(law, nA, nB, lambd1, lambd2, P_A1, P_A2, P_B1, P_B2, M_inv, direction)

(0.020592808746121988, 0.013684715696249017, 0.013099405751815008)

In [17]:
#Number of repetitions
K = 1000

In [18]:
List_KS1, List_KSA, List_KSB = RunParallelSimulation(law, nA, nB, lambd1, lambd2, P_A1, P_A2, P_B1, P_B2, M_inv, direction, K, NJobs=-1, Verbose=0, PreDispatch='2*n_jobs')

In [19]:
List_KS1 = List_KS1.tolist()
List_KSA = List_KSA.tolist()
List_KSB = List_KSB.tolist()

https://real-statistics.com/statistics-tables/kolmogorov-smirnov-table/

In [20]:
t_10 = 1.22385
t_5 = 1.35810

In [21]:
s_alpha_A = 2*sqrt(2)*t_5*(np.abs(M_inv[0][0]))
s_alpha_A 

5.761930317176702

In [22]:
s_alpha_B = 2*sqrt(2)*t_5*(np.abs(M_inv[0][1]))
s_alpha_B 

1.9206434390589002

In [23]:
s = 0
for i in range(K):
    if List_KSA[i] > t_5/np.sqrt(nA):
        s+=1
sa = s/K
sa

0.053

In [24]:
s = 0
for i in range(K):
    if List_KSB[i] > t_5/np.sqrt(nB):
        s+=1
sb = s/K
sb

0.045

In [25]:
s_alpha = max(s_alpha_A,s_alpha_B)
s_alpha

5.761930317176702

In [26]:
s = 0
for i in range(K):
    if List_KS1[i] > s_alpha/np.sqrt(n):
        s+=1
s1 = s/K
s1

0.0